In [1]:
import os
import json
def loads_json(file):
    with open(file, 'r') as f:
        data = json.loads(f.read())
    return data

eval_output_dir = '/home/ubuntu/project/OpenHands/evaluation/evaluation_outputs/outputs'
output_file = 'openhands/use_code_hint/princeton-nlp__SWE-bench_Lite-test/CodeActAgent/claude-3-5-sonnet-20241022_maxiter_30_N_v2.1-no-hint-run_1/output.jsonl'
eval_output_file = os.path.join(eval_output_dir, output_file)

In [3]:
gt_file = '/home/ubuntu/auto-search-agent/evaluation/gt_data/SWE-bench_Lite/gt_locs_data_20241030.json'
gt_datas = loads_json(gt_file)

In [45]:
import re

# Creating a mapping of file paths to their corresponding functions
from collections import defaultdict
def is_test(name, test_phrases=None):
    if test_phrases is None:
        test_phrases = ['test', 'tests', 'testing']
    words = set(re.split(r' |_|\/|\.', name.lower()))
    return any(word in words for word in test_phrases)

def extract_locations(code_str):
    # Initialize a defaultdict for easier management of lists
    file_function_map = defaultdict(list)

    # Extracting paths and functions again to build the desired structure
    matches = re.finditer(r"'path':\s*'([^']+)'.*?(def\s+(\w+)\()", code_str, re.DOTALL)

    for match in matches:
        file_path = match.group(1)
        if is_test(file_path) or 'reproduce' in file_path: continue
        # file_path = file_path.split('/')[2:]
        function_name = match.group(3)
        file_function_map[file_path].append(function_name)

    # Ensure all file paths are present in the result even if no functions were found
    all_file_paths = re.findall(r"'path':\s*'([^']+)'", code_str)
    for path in all_file_paths:
        if is_test(path) or 'reproduce' in path: continue
        if path not in file_function_map:
            file_function_map[path] = []

    # Converting defaultdict to a regular dictionary for final result
    result = dict(file_function_map)
    return result

def merge_dict(dict_1, dict_2):
    merged_dict = {}
    for key in set(dict_1) | set(dict_2):  # Using union of keys from both dictionaries
        merged_dict[key] = list(set(dict_1.get(key, []) + dict_2.get(key, [])))
    
    return merged_dict

In [46]:
all_loc_dict = dict()
with open(eval_output_file, 'r') as f:
    for line in f:
        output = json.loads(line)
        instance_id = output['instance_id']
        msg_history = output['history']
        all_loc_dict[instance_id] = dict()
        for i, msg in enumerate(msg_history):
            if i==0: continue
            if 'args' in msg:
                if 'code' in msg['args']:
                    code = msg['args']['code']
                elif 'command' in msg['args']:
                    code = msg['args']['command']
                extracted_loc = extract_locations(code)
                all_loc_dict[instance_id] = merge_dict(all_loc_dict[instance_id], extracted_loc)
        # break
all_loc_dict
# collect code(-> filenames / module names)

{'astropy__astropy-12907': {'/workspace/astropy__astropy__4.3/astropy/modeling/separable.py': ['_separable']},
 'astropy__astropy-14995': {'/workspace/astropy__astropy__5.2/astropy/nddata/mixins/ndarithmetic.py': ['_arithmetic_mask']},
 'astropy__astropy-6938': {'/workspace/astropy__astropy__1.3/astropy/io/fits/fitsrec.py': []},
 'django__django-10914': {'/workspace/django__django__3.0/django/conf/global_settings.py': []},
 'astropy__astropy-14182': {'/workspace/astropy__astropy__5.1/astropy/io/ascii/rst.py': ['write',
   '__init__']},
 'astropy__astropy-14365': {'/workspace/astropy__astropy__5.1/astropy/io/ascii/qdp.py': []},
 'astropy__astropy-7746': {'/workspace/astropy__astropy__1.3/astropy/wcs/wcs.py': ['_return_single_array',
   '_return_list_of_arrays']},
 'django__django-10924': {'/workspace/django__django__3.0/django/db/models/fields/__init__.py': ['deconstruct',
   'formfield',
   '__init__'],
  '/workspace/django__django__3.0/django/forms/fields.py': []},
 'django__django-11

In [ ]:
import re
# Adjusted input text to handle line continuation characters and simplify extraction logic

# Trimming down and using cleaned strings for better handling
input_text = """
print(file_editor(**{'command': 'create', 'path': '/workspace/reproduce.py'}))
print(file_editor(**{'command': 'str_replace', 'path': '/workspace/astropy__astropy__4.3/astropy/modeling/separable.py', 'old_str': 'def _separable(transform):'}))
"""

# Extracting file paths
file_paths = re.findall(r"'path':\s*'([^']+)'", input_text)

# Extracting function names/modules
function_names = re.findall(r'def\s+(\w+)\(', input_text)

(file_paths, function_names)

(['/workspace/reproduce.py',
  '/workspace/astropy__astropy__4.3/astropy/modeling/separable.py'],
 ['_separable'])

In [25]:
text = input_text.strip().split('\n')[0]
extract_locations(text)

{'/workspace/reproduce.py': []}

In [20]:
matches = re.finditer(r"'path':\s*'([^']+)'.*?(def\s+(\w+)\()", text, re.DOTALL)
for match in matches:
    file_path = match.group(1)
    function_name = match.group(3)
    # file_function_map[file_path].append(function_name)
    print(file_path)